In [7]:
from IPython.display import display, HTML,clear_output
# css = """
# .output {
# flex-direction: row;
# flex-wrap: wrap;
# }
# """

# HTML('<style>{}</style>'.format(css))

ERROR! Session/line number was not unique in database. History logging moved to new session 186


In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
# %pip install --no-cache-dir matplotlib

# from pandasql import sqldf
import requests
import brotli
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [10]:
pd.options.mode.copy_on_write = True

In [11]:
origin_url="https://www.nseindia.com/market-data/most-active-contracts"
url = 'https://www.nseindia.com/api/snapshot-derivatives-equity?index=oi'
default_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

header = {
             "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
             "Accept-Language": "en-US,en;q=0.9,hi;q=0.8"
            }


session = requests.session()
nse_live = session.get(origin_url, headers=default_header)
cookies = nse_live.cookies
session.get(url, headers=header, cookies=cookies)


<Response [200]>

In [12]:
columns = [
    "underlying", "expiryDate", "optionType", "strikePrice", "lastPrice", "numberOfContractsTraded",
    "totalTurnover", "openInterest", "underlyingValue", "pChange", "timestamp"
]
# Create an empty DataFrame
filter_df = pd.DataFrame(columns=columns)

In [12]:
def main_dis_run(filter_df):
  try:
    while True:
      clear_output(wait=True)
      data = session.get(url,headers=header).json()['value']['data']
      df = pd.DataFrame(data)
      df.drop(columns =['identifier','instrumentType','instrument','premiumTurnover','numberOfContractsTraded','totalTurnover','pChange'],inplace=True)
      df_put = df[(df['underlying'] == 'NIFTY') & (df['optionType'].isin(['Put','-']))].sort_values(by=['strikePrice', 'openInterest'], ascending=True)
      df_call = df[(df['underlying'] == 'NIFTY') & (df['optionType'] == 'Call')].sort_values(by=['strikePrice', 'openInterest'], ascending=True)
      df_call.columns = ['C_' + col for col in df_call.columns ]
      df_call = df_call.drop(columns=['C_underlying','C_underlyingValue'])
      df_call['strikePrice'] = df_call['C_strikePrice']
      df_call.drop(columns=['C_strikePrice'])
      df_merged = pd.merge(df_put, df_call, left_on='strikePrice', right_on='strikePrice', how='outer')

      df_merged['expiryDate'] = df_merged['expiryDate'].combine_first(df_merged['C_expiryDate'])
      df_merged['optionType'] = df_merged['optionType'].combine_first(df_merged['C_optionType'])
      df_merged['lastPrice'] = df_merged['lastPrice'].combine_first(df_merged['C_lastPrice'])
      df_merged = df_merged.drop(columns = ['C_expiryDate','C_optionType','C_strikePrice','C_lastPrice'])

      df_merged['underlying'] = df_merged['underlying'].fillna('NIFTY')

      # Fill missing 'underlyingValue' with the first non-null value
      first_value = df_merged['underlyingValue'].dropna().iloc[0]
      df_merged['underlyingValue'] = df_merged['underlyingValue'].fillna(first_value)
      display(df_merged)
      time.sleep(5)
  except Exception as e:
      clear_output(wait=True)
      print(f'trying to retry to re-slove issue')
      # raise e
      main_dis_run(filter_df)

main_dis_run(filter_df)

trying to retry to re-slove issue


In [ ]:
# df = pd.read_csv('D:/badri/OptionChainData/nifty/nifty.csv')
# df_unique = df.drop_duplicates(subset=df.columns.difference(['timestamp']), keep='last')